# Brick Stack with Network Graph Data Structure

This notebook demonstrates how to represent architectural elements using a **network graph** where:
- **Nodes** = Individual bricks (building elements)
- **Edges** = Relationships between bricks (mortar connections)

This approach enables computational analysis of building assemblies and their connections.

## Setup: Import Libraries and Initialize Network

Import COMPAS libraries and create a **Network** data structure to represent brick relationships.

In [ ]:
import compas.geometry as cg
from compas_notebook.viewer import Viewer
import compas.datastructures as cd
# Create a viewer
viewer = Viewer()

network = cd.Network()

BRICK_DIM = [0.230, 0.110, 0.050]
MORTAR_THICKNESS = 0.010
LAYER_HEIGHT = BRICK_DIM[2] + MORTAR_THICKNESS
MAXIMUM_INTERSECTION_DISTANCE = (BRICK_DIM[0]**2 + BRICK_DIM[1]**2 + BRICK_DIM[2]**2) ** 0.5 - 0.01

## Create Bricks as Network Nodes

### Why Store Bricks as Nodes?
**Nodes represent discrete building elements** - each brick is an individual component with:
- Geometric properties (shape, position, orientation)
- Material attributes (could be added later)
- Structural properties (layer, ID, coordinates)

This allows us to query, analyze, and modify individual bricks within the assembly.

In [ ]:
# Define two stacked bricks with different positions and orientations
frame1 = cg.Frame([0, 0, 0], [1, 0, 0], [0, 1, 0])
frame2 = cg.Frame([0, 0.01, LAYER_HEIGHT], [1, 1, 0], [0, 1, 0])

brick_1 = cg.Box(BRICK_DIM[0], BRICK_DIM[1], BRICK_DIM[2], frame1)
brick_2 = cg.Box(BRICK_DIM[0], BRICK_DIM[1], BRICK_DIM[2], frame2)

# Add first brick as a network node with attributes
brick_1_key = network.add_node(
    attr_dict={
        'geometry': brick_1,           # Store the actual geometry
        "x": brick_1.frame.point.x,    # X coordinate for analysis
        "y": brick_1.frame.point.y,    # Y coordinate for analysis  
        "z": brick_1.frame.point.z,    # Z coordinate for analysis
        "layer": 0,                    # Building layer/course number
        "brick_type": "bottom"         # Custom attribute for brick role
    } 
)

# Add second brick as a network node with attributes
brick_2_key = network.add_node(
    attr_dict={
        'geometry': brick_2,
        "x": brick_2.frame.point.x,
        "y": brick_2.frame.point.y,
        "z": brick_2.frame.point.z,
        "layer": 1,
        "brick_type": "top"
    }
)

# Create an edge connecting the two bricks (representing their relationship)
edge = network.add_edge(brick_1_key, brick_2_key)

print(f"✅ Created network with {network.number_of_nodes()} nodes and {network.number_of_edges()} edges")
print(f"🧱 Brick 1 (Node {brick_1_key}): Layer {network.node_attribute(brick_1_key, 'layer')}")
print(f"🧱 Brick 2 (Node {brick_2_key}): Layer {network.node_attribute(brick_2_key, 'layer')}")

# Visualize the network (shows both geometry and graph structure)
viewer.scene.add(network)
viewer.show()

## Define Mortar Computation Function

Create a function to compute the mortar volume between two bricks using boolean intersection of enlarged brick volumes.

### Boolean Intersection Process:
1. **Enlarge both bricks** by mortar thickness
2. **Find intersection** of enlarged volumes  
3. **Result** = mortar space between original bricks

In [ ]:
def compute_mortar(brick_1, brick_2):
    """
    Compute the mortar between two brickes and return as a mesh.
    """

    # instersection is computational expensive, so we comute it only when brickes are close enough
    distance = brick_1.frame.point.distance_to_point(brick_2.frame.point)
    if distance > MAXIMUM_INTERSECTION_DISTANCE:
        return None


    # generate larger brick for boolean intersection
    brick_1_large = cg.Box(
        BRICK_DIM[0],
        BRICK_DIM[1],
        BRICK_DIM[2] + MORTAR_THICKNESS*2,
        brick_1.frame
    )
    brick_2_large = cg.Box(
        BRICK_DIM[0],
        BRICK_DIM[1],
        BRICK_DIM[2] + MORTAR_THICKNESS*2,
        brick_2.frame
    )

    # boolean intersection takes vertices and faces as input
    brick_1_vf = brick_1_large.to_mesh(True, 10, 10).to_vertices_and_faces()
    brick_2_vf = brick_2_large.to_mesh(True, 10, 10).to_vertices_and_faces()

    # perform boolean intersection
    intersection = cg.boolean_intersection_mesh_mesh(brick_1_vf, brick_2_vf)

    # convert result to mesh
    intersection_mesh = cd.Mesh.from_vertices_and_faces(*intersection)
    
    return intersection_mesh
    

## Store Mortar as Edge Attribute

### Why Store Mortar as Edge Attributes?
**Edges represent relationships between elements** - mortar is the physical connection between two bricks:
- **Connection-specific**: Each mortar joint is unique to a brick pair
- **Relational data**: Mortar properties depend on both connected bricks  
- **Network analysis**: Enables structural analysis of connections
- **Efficient queries**: Easy to find all connections for any brick

This approach mirrors real construction where mortar joints connect discrete masonry units.

In [ ]:
# Compute mortar between the two connected bricks
mortar = compute_mortar(brick_1, brick_2)

# Store the mortar geometry as an attribute of the edge connecting the bricks
network.edge_attribute(edge, 'mortar', mortar)


# Add mortar to visualization
viewer.scene.add(mortar, color=(200, 200, 200), opacity=0.6)

# Display the complete assembly
viewer.show()

# Demonstrate network queries
print(f"\n📋 Network Analysis:")
print(f"   • Total nodes (bricks): {network.number_of_nodes()}")
print(f"   • Total edges (connections): {network.number_of_edges()}")
print(f"   • Neighbors of brick {brick_1_key}: {list(network.neighbors(brick_1_key))}")
print(f"   • Neighbors of brick {brick_2_key}: {list(network.neighbors(brick_2_key))}")

# Query edge attributes
mortar_data = network.edge_attribute(edge, 'mortar')
connection_type = network.edge_attribute(edge, 'connection_type')
print(f"   • Connection type: {connection_type}")
print(f"   • Mortar data stored: {'Yes' if mortar_data else 'No'}")

## Summary: Graph-Based Building Assembly

### Key Concepts Demonstrated:

**🏗️ Architectural Graph Modeling:**
- **Nodes** = Discrete building elements (bricks, stones, blocks)
- **Edges** = Physical connections (mortar, adhesive, mechanical fasteners)

**📊 Advantages of This Approach:**
1. **Structural Analysis**: Query connectivity patterns and load paths
2. **Material Quantification**: Calculate total mortar volumes efficiently  
3. **Quality Control**: Identify missing or weak connections
4. **Parametric Design**: Modify assemblies while preserving relationships
5. **Simulation Ready**: Network structure enables structural and thermal analysis

**🔍 Real-World Applications:**
- Masonry wall analysis and optimization
- Prefab construction planning
- Building information modeling (BIM)
- Structural health monitoring
- Construction sequencing

This graph-based approach transforms building assemblies into computable networks for advanced architectural analysis.